# Sequence-to-Sequence Translation Experiments

This notebook implements various encoder-decoder architectures for French-to-English translation:

1. **Baseline**: GRU Encoder + GRU Decoder
2. **Experiment 1**: LSTM Encoder + LSTM Decoder
3. **Experiment 2**: Bi-LSTM Encoder + GRU Decoder
4. **Experiment 3**: GRU Encoder + GRU Decoder with Attention
5. **Experiment 4**: Transformer Encoder + GRU Decoder

We will record and compare Rouge scores for each model.

## Setup and Imports

In [ ]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np

from sklearn.model_selection import train_test_split
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Data Preparation

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    """Language vocabulary class for mapping words to indices and vice versa."""
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def unicodeToAscii(s):
    """Convert Unicode string to plain ASCII."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    """Lowercase, trim, and remove non-letter characters."""
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    """Read translation pairs from file."""
    print("Reading lines...")
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)

def filterPair(p):
    """Filter pairs by length and prefix."""
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    """Prepare data: read, filter, and build vocabulary."""
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(f"\nExample pair: {random.choice(pairs)}")

In [ ]:
# Train/test split
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

train_pairs = list(zip(X_train, y_train))
test_pairs = list(zip(X_test, y_test))

print(f"Training pairs: {len(train_pairs)}")
print(f"Test pairs: {len(test_pairs)}")

## Helper Functions

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## Evaluation Functions

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, use_attention=False):
    """Evaluate a single sentence."""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            if use_attention:
                decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def inference(encoder, decoder, testing_pairs, use_attention=False):
    """Run inference on test pairs."""
    inputs = []
    gt = []
    predict = []

    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate(encoder, decoder, pair[0], use_attention=use_attention)
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return inputs, gt, predict

def eval_rouge(gt, predict):
    """Calculate and print Rouge scores."""
    rouge = ROUGEScore()
    metric_score = rouge(predict, gt)
    print("=" * 45)
    print("         Evaluation Score - ROUGE Score")
    print("=" * 45)
    print(f"Rouge1 F-measure:  {metric_score['rouge1_fmeasure'].item():.4f}")
    print(f"Rouge1 Precision:  {metric_score['rouge1_precision'].item():.4f}")
    print(f"Rouge1 Recall:     {metric_score['rouge1_recall'].item():.4f}")
    print(f"Rouge2 F-measure:  {metric_score['rouge2_fmeasure'].item():.4f}")
    print(f"Rouge2 Precision:  {metric_score['rouge2_precision'].item():.4f}")
    print(f"Rouge2 Recall:     {metric_score['rouge2_recall'].item():.4f}")
    print("=" * 45)
    return metric_score

In [ ]:
# Dictionary to store results
results = {}

---
# Task 2: Baseline - GRU Encoder + GRU Decoder

This is the original implementation using GRU units for both encoder and decoder.

In [ ]:
class EncoderGRU(nn.Module):
    """GRU-based Encoder."""
    def __init__(self, input_size, hidden_size):
        super(EncoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class DecoderGRU(nn.Module):
    """GRU-based Decoder."""
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5

def train_step(input_tensor, target_tensor, encoder, decoder, 
               encoder_optimizer, decoder_optimizer, criterion, 
               max_length=MAX_LENGTH, use_attention=False):
    """Single training step."""
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            if use_attention:
                decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            if use_attention:
                decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(encoder, decoder, epochs, print_every=1000, 
               learning_rate=0.01, use_attention=False):
    """Training loop."""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    iter_count = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_step(input_tensor, target_tensor, encoder, decoder,
                             encoder_optimizer, decoder_optimizer, criterion,
                             use_attention=use_attention)
            print_loss_total += loss

            if iter_count % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                            iter_count, iter_count / n_iters * 100, print_loss_avg))
            iter_count += 1
    
    print(f"\nTraining completed in {asMinutes(time.time() - start)}")

In [ ]:
# Train Baseline: GRU Encoder + GRU Decoder
print("="*60)
print("BASELINE: GRU Encoder + GRU Decoder")
print("="*60)

hidden_size = 256
encoder_gru = EncoderGRU(input_lang.n_words, hidden_size).to(device)
decoder_gru = DecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters(encoder_gru, decoder_gru, epochs=5, print_every=5000)

In [ ]:
# Evaluate Baseline
print("\nEvaluating Baseline (GRU + GRU)...")
_, gt, predict = inference(encoder_gru, decoder_gru, test_pairs)
results['Baseline_GRU'] = eval_rouge(gt, predict)

---
# Task 3: LSTM Encoder + LSTM Decoder

Replace GRU with LSTM in both encoder and decoder.

In [ ]:
class EncoderLSTM(nn.Module):
    """LSTM-based Encoder."""
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        # LSTM requires (h_0, c_0)
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))


class DecoderLSTM(nn.Module):
    """LSTM-based Decoder."""
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
def train_step_lstm(input_tensor, target_tensor, encoder, decoder, 
                    encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """Training step for LSTM models."""
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters_lstm(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    """Training loop for LSTM models."""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    iter_count = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_step_lstm(input_tensor, target_tensor, encoder, decoder,
                                   encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter_count % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                            iter_count, iter_count / n_iters * 100, print_loss_avg))
            iter_count += 1
    
    print(f"\nTraining completed in {asMinutes(time.time() - start)}")

In [ ]:
def evaluate_lstm(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """Evaluate for LSTM models."""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words


def inference_lstm(encoder, decoder, testing_pairs):
    """Inference for LSTM models."""
    inputs = []
    gt = []
    predict = []

    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate_lstm(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return inputs, gt, predict

In [ ]:
# Train LSTM Encoder + LSTM Decoder
print("="*60)
print("EXPERIMENT 1: LSTM Encoder + LSTM Decoder")
print("="*60)

encoder_lstm = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder_lstm = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

trainIters_lstm(encoder_lstm, decoder_lstm, epochs=5, print_every=5000)

In [ ]:
# Evaluate LSTM
print("\nEvaluating LSTM Encoder + LSTM Decoder...")
_, gt, predict = inference_lstm(encoder_lstm, decoder_lstm, test_pairs)
results['LSTM_LSTM'] = eval_rouge(gt, predict)

---
# Task 4: Bi-LSTM Encoder + GRU Decoder

Replace the encoder with Bidirectional LSTM while keeping GRU decoder.

In [ ]:
class EncoderBiLSTM(nn.Module):
    """Bidirectional LSTM Encoder."""
    def __init__(self, input_size, hidden_size):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        # Project bidirectional output back to hidden_size
        self.fc = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        # Combine forward and backward outputs
        output = self.fc(output)
        # Combine forward and backward hidden states
        h_n = hidden[0]  # (2, 1, hidden_size)
        c_n = hidden[1]  # (2, 1, hidden_size)
        # Concatenate and project back
        h_combined = torch.cat((h_n[0], h_n[1]), dim=1).unsqueeze(0)  # (1, 1, hidden_size*2)
        h_combined = self.fc(h_combined)
        return output, h_combined

    def initHidden(self):
        # Bidirectional LSTM: 2 directions
        return (torch.zeros(2, 1, self.hidden_size, device=device),
                torch.zeros(2, 1, self.hidden_size, device=device))

In [ ]:
def train_step_bilstm(input_tensor, target_tensor, encoder, decoder, 
                      encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """Training step for BiLSTM encoder + GRU decoder."""
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder.initHidden())
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    # encoder_hidden is already projected to correct size
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters_bilstm(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    """Training loop for BiLSTM encoder."""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    iter_count = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_step_bilstm(input_tensor, target_tensor, encoder, decoder,
                                     encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter_count % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                            iter_count, iter_count / n_iters * 100, print_loss_avg))
            iter_count += 1
    
    print(f"\nTraining completed in {asMinutes(time.time() - start)}")

In [ ]:
def evaluate_bilstm(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """Evaluate for BiLSTM encoder + GRU decoder."""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder.initHidden())
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words


def inference_bilstm(encoder, decoder, testing_pairs):
    """Inference for BiLSTM encoder."""
    inputs = []
    gt = []
    predict = []

    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate_bilstm(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return inputs, gt, predict

In [ ]:
# Train Bi-LSTM Encoder + GRU Decoder
print("="*60)
print("EXPERIMENT 2: Bi-LSTM Encoder + GRU Decoder")
print("="*60)

encoder_bilstm = EncoderBiLSTM(input_lang.n_words, hidden_size).to(device)
decoder_gru2 = DecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters_bilstm(encoder_bilstm, decoder_gru2, epochs=5, print_every=5000)

In [ ]:
# Evaluate Bi-LSTM
print("\nEvaluating Bi-LSTM Encoder + GRU Decoder...")
_, gt, predict = inference_bilstm(encoder_bilstm, decoder_gru2, test_pairs)
results['BiLSTM_GRU'] = eval_rouge(gt, predict)

---
# Task 5: GRU Encoder + GRU Decoder with Attention

Add the attention mechanism between encoder and decoder.

## Attention Mechanism

At each decoder timestep $t$:

1. **Compute attention scores**: $e_{t,i} = score(s_t, h_i)$
2. **Normalize**: $\alpha_{t,i} = softmax(e_{t,i})$
3. **Context vector**: $c_t = \sum_i \alpha_{t,i} \cdot h_i$
4. **Combine**: $\hat{s}_t = concat(s_t, c_t)$

In [ ]:
class AttnDecoderGRU(nn.Module):
    """
    GRU Decoder with Bahdanau-style Attention.
    
    Attention Mechanism:
    - Query: decoder hidden state s_t
    - Keys/Values: encoder hidden states h_i
    - Score function: additive (Bahdanau): v^T * tanh(W_s * s_t + W_h * h_i)
    """
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH, dropout_p=0.1):
        super(AttnDecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        
        # Attention layers (Bahdanau/Additive attention)
        self.attn_W_s = nn.Linear(hidden_size, hidden_size)  # For decoder state
        self.attn_W_h = nn.Linear(hidden_size, hidden_size)  # For encoder states
        self.attn_v = nn.Linear(hidden_size, 1)  # For computing scalar score
        
        # GRU takes embedded input
        self.gru = nn.GRU(hidden_size, hidden_size)
        
        # Output layer takes concatenated [s_t; c_t]
        self.out = nn.Linear(hidden_size * 2, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        """
        Forward pass with attention.
        
        Args:
            input: Current input token (1,)
            hidden: Previous decoder hidden state (1, 1, hidden_size)
            encoder_outputs: All encoder hidden states (max_length, hidden_size)
        
        Returns:
            output: Log probabilities over vocabulary
            hidden: Updated hidden state
            attn_weights: Attention weights for visualization
        """
        # Step 1: Embed input
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        # Step 2: Run GRU to get decoder state s_t
        gru_output, hidden = self.gru(embedded, hidden)
        s_t = hidden.squeeze(0)  # (1, hidden_size)
        
        # Step 3: Compute attention scores using Bahdanau (additive) attention
        # e_{t,i} = v^T * tanh(W_s * s_t + W_h * h_i)
        
        # Transform decoder state: W_s * s_t -> (1, hidden_size)
        s_t_transformed = self.attn_W_s(s_t)  # (1, hidden_size)
        
        # Transform encoder outputs: W_h * h_i -> (max_length, hidden_size)
        h_transformed = self.attn_W_h(encoder_outputs)  # (max_length, hidden_size)
        
        # Add and apply tanh: (max_length, hidden_size)
        energy = torch.tanh(s_t_transformed + h_transformed)
        
        # Compute scalar scores: v^T * energy -> (max_length, 1)
        attn_scores = self.attn_v(energy).squeeze(-1)  # (max_length,)
        
        # Step 4: Normalize scores to get attention distribution
        attn_weights = F.softmax(attn_scores, dim=0)  # (max_length,)
        
        # Step 5: Compute context vector c_t = sum(alpha * h_i)
        context = torch.sum(attn_weights.unsqueeze(1) * encoder_outputs, dim=0)  # (hidden_size,)
        context = context.unsqueeze(0)  # (1, hidden_size)
        
        # Step 6: Combine decoder state with context: [s_t; c_t]
        combined = torch.cat((s_t, context), dim=1)  # (1, hidden_size * 2)
        
        # Step 7: Compute output probabilities
        output = self.softmax(self.out(combined))  # (1, output_size)
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def train_step_attention(input_tensor, target_tensor, encoder, decoder, 
                         encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """Training step with attention."""
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    # Encode all input tokens
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters_attention(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    """Training loop with attention."""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    iter_count = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_step_attention(input_tensor, target_tensor, encoder, decoder,
                                        encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter_count % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                            iter_count, iter_count / n_iters * 100, print_loss_avg))
            iter_count += 1
    
    print(f"\nTraining completed in {asMinutes(time.time() - start)}")

In [ ]:
def evaluate_attention(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """Evaluate with attention."""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words


def inference_attention(encoder, decoder, testing_pairs):
    """Inference with attention."""
    inputs = []
    gt = []
    predict = []

    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate_attention(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return inputs, gt, predict

In [ ]:
# Train GRU Encoder + Attention Decoder
print("="*60)
print("EXPERIMENT 3: GRU Encoder + GRU Decoder with Attention")
print("="*60)

encoder_gru_attn = EncoderGRU(input_lang.n_words, hidden_size).to(device)
decoder_attn = AttnDecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters_attention(encoder_gru_attn, decoder_attn, epochs=5, print_every=5000)

In [ ]:
# Evaluate Attention
print("\nEvaluating GRU Encoder + Attention Decoder...")
_, gt, predict = inference_attention(encoder_gru_attn, decoder_attn, test_pairs)
results['GRU_Attention'] = eval_rouge(gt, predict)

---
# Task 6: Transformer Encoder + GRU Decoder

Replace the GRU encoder with a Transformer encoder. The sentence representation is the mean of all token representations.

In [ ]:
class PositionalEncoding(nn.Module):
    """Positional encoding for Transformer."""
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class TransformerEncoder(nn.Module):
    """
    Transformer Encoder.
    
    Processes the entire input sequence at once and returns:
    - A sentence representation (mean of all token representations)
    - All encoder outputs for potential attention
    """
    def __init__(self, input_size, hidden_size, nhead=4, num_layers=2, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size, dropout=dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size, 
            nhead=nhead, 
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Project to hidden size for decoder compatibility
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, input_tensor):
        """
        Forward pass - processes entire sequence at once.
        
        Args:
            input_tensor: Input indices (seq_len, 1)
            
        Returns:
            sentence_repr: Mean of all token representations (1, 1, hidden_size)
            encoder_outputs: All encoder outputs (seq_len, hidden_size)
        """
        # Embed: (seq_len, 1) -> (seq_len, 1, hidden_size)
        embedded = self.embedding(input_tensor.squeeze(-1)).unsqueeze(1)
        
        # Add positional encoding
        embedded = self.pos_encoder(embedded)
        
        # Transformer encoding: (seq_len, 1, hidden_size)
        transformer_output = self.transformer_encoder(embedded)
        
        # Get sentence representation as mean of all tokens
        # (seq_len, 1, hidden_size) -> (1, hidden_size)
        sentence_repr = transformer_output.mean(dim=0)
        sentence_repr = self.fc(sentence_repr)
        sentence_repr = sentence_repr.unsqueeze(0)  # (1, 1, hidden_size)
        
        # Encoder outputs for each position: (seq_len, hidden_size)
        encoder_outputs = transformer_output.squeeze(1)  # (seq_len, hidden_size)
        
        return sentence_repr, encoder_outputs

    def initHidden(self):
        # Not used for Transformer, but kept for API compatibility
        return None

In [ ]:
def train_step_transformer(input_tensor, target_tensor, encoder, decoder, 
                           encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """Training step for Transformer encoder."""
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0

    # Encode entire sequence at once
    encoder_hidden, encoder_outputs_raw = encoder(input_tensor)
    
    # Pad encoder outputs to max_length
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    for ei in range(min(input_length, max_length)):
        encoder_outputs[ei] = encoder_outputs_raw[ei]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters_transformer(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    """Training loop for Transformer encoder."""
    start = time.time()
    print_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate * 0.1)  # Lower LR for Transformer
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    iter_count = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_step_transformer(input_tensor, target_tensor, encoder, decoder,
                                          encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter_count % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                            iter_count, iter_count / n_iters * 100, print_loss_avg))
            iter_count += 1
    
    print(f"\nTraining completed in {asMinutes(time.time() - start)}")

In [ ]:
def evaluate_transformer(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """Evaluate for Transformer encoder."""
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        
        # Encode entire sequence
        encoder_hidden, _ = encoder(input_tensor)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words


def inference_transformer(encoder, decoder, testing_pairs):
    """Inference for Transformer encoder."""
    inputs = []
    gt = []
    predict = []

    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate_transformer(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return inputs, gt, predict

In [ ]:
# Train Transformer Encoder + GRU Decoder
print("="*60)
print("EXPERIMENT 4: Transformer Encoder + GRU Decoder")
print("="*60)

encoder_transformer = TransformerEncoder(input_lang.n_words, hidden_size, nhead=4, num_layers=2).to(device)
decoder_gru_trans = DecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters_transformer(encoder_transformer, decoder_gru_trans, epochs=5, print_every=5000)

In [ ]:
# Evaluate Transformer
print("\nEvaluating Transformer Encoder + GRU Decoder...")
_, gt, predict = inference_transformer(encoder_transformer, decoder_gru_trans, test_pairs)
results['Transformer_GRU'] = eval_rouge(gt, predict)

---
# Results Summary

Compare all models' performance.

In [ ]:
import pandas as pd

# Create results summary table
summary_data = []

for model_name, scores in results.items():
    summary_data.append({
        'Model': model_name,
        'Rouge-1 F1': f"{scores['rouge1_fmeasure'].item():.4f}",
        'Rouge-1 Precision': f"{scores['rouge1_precision'].item():.4f}",
        'Rouge-1 Recall': f"{scores['rouge1_recall'].item():.4f}",
        'Rouge-2 F1': f"{scores['rouge2_fmeasure'].item():.4f}",
        'Rouge-2 Precision': f"{scores['rouge2_precision'].item():.4f}",
        'Rouge-2 Recall': f"{scores['rouge2_recall'].item():.4f}"
    })

results_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("                         RESULTS SUMMARY")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

In [ ]:
# Visualize results
import matplotlib.pyplot as plt

models = list(results.keys())
rouge1_f1 = [results[m]['rouge1_fmeasure'].item() for m in models]
rouge2_f1 = [results[m]['rouge2_fmeasure'].item() for m in models]

x = np.arange(len(models))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, rouge1_f1, width, label='Rouge-1 F1', color='steelblue')
bars2 = ax.bar(x + width/2, rouge2_f1, width, label='Rouge-2 F1', color='coral')

ax.set_ylabel('F1 Score')
ax.set_title('Model Comparison - Rouge Scores')
ax.set_xticks(x)
ax.set_xticklabels(models, rotation=15, ha='right')
ax.legend()
ax.set_ylim(0, 1)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150)
plt.show()

---
# Detailed Results Analysis and Report

This section provides comprehensive analysis, comparison, and explanation of the experimental findings.

## Table 1: Complete Rouge Score Comparison

This table summarizes all Rouge metrics across all model architectures.

In [ ]:
# Table 1: Comprehensive Results Summary Table
from IPython.display import display, HTML

def create_results_table(results):
    """Create a formatted HTML table of all results."""
    
    # Extract data
    table_data = []
    for model_name, scores in results.items():
        table_data.append({
            'Model': model_name.replace('_', ' '),
            'Rouge-1 F1': scores['rouge1_fmeasure'].item(),
            'Rouge-1 Prec': scores['rouge1_precision'].item(),
            'Rouge-1 Rec': scores['rouge1_recall'].item(),
            'Rouge-2 F1': scores['rouge2_fmeasure'].item(),
            'Rouge-2 Prec': scores['rouge2_precision'].item(),
            'Rouge-2 Rec': scores['rouge2_recall'].item(),
        })
    
    df = pd.DataFrame(table_data)
    
    # Find best values for highlighting
    best_r1_f1 = df['Rouge-1 F1'].max()
    best_r2_f1 = df['Rouge-2 F1'].max()
    
    # Create styled dataframe
    def highlight_best(val, best_val):
        if val == best_val:
            return 'background-color: #90EE90; font-weight: bold'
        return ''
    
    # Format to 4 decimal places
    df_display = df.copy()
    for col in df.columns[1:]:
        df_display[col] = df[col].apply(lambda x: f'{x:.4f}')
    
    print("="*100)
    print("                              TABLE 1: COMPLETE ROUGE SCORE COMPARISON")
    print("="*100)
    display(df_display)
    
    return df

results_df_full = create_results_table(results)

## Table 2: Model Ranking by Performance

Ranking models based on Rouge-1 and Rouge-2 F1 scores.

In [ ]:
# Table 2: Model Ranking Table
def create_ranking_table(results):
    """Create a ranking table based on F1 scores."""
    
    ranking_data = []
    for model_name, scores in results.items():
        ranking_data.append({
            'Model': model_name.replace('_', ' '),
            'Rouge-1 F1': scores['rouge1_fmeasure'].item(),
            'Rouge-2 F1': scores['rouge2_fmeasure'].item(),
        })
    
    df = pd.DataFrame(ranking_data)
    
    # Add rankings
    df['Rouge-1 Rank'] = df['Rouge-1 F1'].rank(ascending=False).astype(int)
    df['Rouge-2 Rank'] = df['Rouge-2 F1'].rank(ascending=False).astype(int)
    df['Average Rank'] = ((df['Rouge-1 Rank'] + df['Rouge-2 Rank']) / 2)
    
    # Sort by average rank
    df = df.sort_values('Average Rank')
    
    # Format scores
    df_display = df.copy()
    df_display['Rouge-1 F1'] = df['Rouge-1 F1'].apply(lambda x: f'{x:.4f}')
    df_display['Rouge-2 F1'] = df['Rouge-2 F1'].apply(lambda x: f'{x:.4f}')
    df_display['Average Rank'] = df['Average Rank'].apply(lambda x: f'{x:.1f}')
    
    print("\n" + "="*80)
    print("                    TABLE 2: MODEL RANKING BY PERFORMANCE")
    print("="*80)
    display(df_display[['Model', 'Rouge-1 F1', 'Rouge-1 Rank', 'Rouge-2 F1', 'Rouge-2 Rank', 'Average Rank']])
    
    return df

ranking_df = create_ranking_table(results)

## Table 3: Performance Improvement Analysis

Comparing each model's improvement over the baseline (GRU + GRU).

In [ ]:
# Table 3: Improvement Over Baseline Analysis
def create_improvement_table(results):
    """Calculate improvement over baseline for each model."""
    
    # Get baseline scores
    baseline_r1 = results['Baseline_GRU']['rouge1_fmeasure'].item()
    baseline_r2 = results['Baseline_GRU']['rouge2_fmeasure'].item()
    
    improvement_data = []
    for model_name, scores in results.items():
        r1_f1 = scores['rouge1_fmeasure'].item()
        r2_f1 = scores['rouge2_fmeasure'].item()
        
        # Calculate percentage improvement
        r1_improvement = ((r1_f1 - baseline_r1) / baseline_r1) * 100
        r2_improvement = ((r2_f1 - baseline_r2) / baseline_r2) * 100
        
        improvement_data.append({
            'Model': model_name.replace('_', ' '),
            'Rouge-1 F1': r1_f1,
            'R1 vs Baseline': r1_improvement,
            'Rouge-2 F1': r2_f1,
            'R2 vs Baseline': r2_improvement,
        })
    
    df = pd.DataFrame(improvement_data)
    
    # Format for display
    df_display = df.copy()
    df_display['Rouge-1 F1'] = df['Rouge-1 F1'].apply(lambda x: f'{x:.4f}')
    df_display['Rouge-2 F1'] = df['Rouge-2 F1'].apply(lambda x: f'{x:.4f}')
    df_display['R1 vs Baseline'] = df['R1 vs Baseline'].apply(lambda x: f'{x:+.2f}%')
    df_display['R2 vs Baseline'] = df['R2 vs Baseline'].apply(lambda x: f'{x:+.2f}%')
    
    print("\n" + "="*90)
    print("                    TABLE 3: PERFORMANCE IMPROVEMENT OVER BASELINE")
    print("="*90)
    print(f"Baseline Model: GRU Encoder + GRU Decoder")
    print(f"Baseline Rouge-1 F1: {baseline_r1:.4f} | Baseline Rouge-2 F1: {baseline_r2:.4f}")
    print("-"*90)
    display(df_display)
    
    return df

improvement_df = create_improvement_table(results)

## Detailed Analysis and Comparison

In [ ]:
# Statistical Analysis and Detailed Comparison
def detailed_analysis(results):
    """Provide detailed statistical analysis of results."""
    
    print("\n" + "="*100)
    print("                              DETAILED ANALYSIS AND COMPARISON")
    print("="*100)
    
    # Extract scores
    models = list(results.keys())
    r1_scores = [results[m]['rouge1_fmeasure'].item() for m in models]
    r2_scores = [results[m]['rouge2_fmeasure'].item() for m in models]
    
    # Statistical summary
    print("\n" + "-"*50)
    print("1. STATISTICAL SUMMARY")
    print("-"*50)
    print(f"\nRouge-1 F1 Scores:")
    print(f"   Mean:     {np.mean(r1_scores):.4f}")
    print(f"   Std Dev:  {np.std(r1_scores):.4f}")
    print(f"   Min:      {np.min(r1_scores):.4f} ({models[np.argmin(r1_scores)]})")
    print(f"   Max:      {np.max(r1_scores):.4f} ({models[np.argmax(r1_scores)]})")
    print(f"   Range:    {np.max(r1_scores) - np.min(r1_scores):.4f}")
    
    print(f"\nRouge-2 F1 Scores:")
    print(f"   Mean:     {np.mean(r2_scores):.4f}")
    print(f"   Std Dev:  {np.std(r2_scores):.4f}")
    print(f"   Min:      {np.min(r2_scores):.4f} ({models[np.argmin(r2_scores)]})")
    print(f"   Max:      {np.max(r2_scores):.4f} ({models[np.argmax(r2_scores)]})")
    print(f"   Range:    {np.max(r2_scores) - np.min(r2_scores):.4f}")
    
    # Best model identification
    best_r1_idx = np.argmax(r1_scores)
    best_r2_idx = np.argmax(r2_scores)
    
    print("\n" + "-"*50)
    print("2. BEST PERFORMING MODELS")
    print("-"*50)
    print(f"\n   Best Rouge-1 F1: {models[best_r1_idx]} ({r1_scores[best_r1_idx]:.4f})")
    print(f"   Best Rouge-2 F1: {models[best_r2_idx]} ({r2_scores[best_r2_idx]:.4f})")
    
    if best_r1_idx == best_r2_idx:
        print(f"\n   ★ Overall Best Model: {models[best_r1_idx]}")
    else:
        print(f"\n   Note: Different models excel at different metrics")
    
    return r1_scores, r2_scores

r1_scores, r2_scores = detailed_analysis(results)

## Pairwise Model Comparison

In [ ]:
# Pairwise Comparison Between Models
def pairwise_comparison(results):
    """Create pairwise comparison analysis."""
    
    print("\n" + "="*100)
    print("                              PAIRWISE MODEL COMPARISON")
    print("="*100)
    
    comparisons = [
        ("Baseline_GRU", "LSTM_LSTM", "GRU vs LSTM", 
         "LSTM includes cell state (memory) in addition to hidden state"),
        ("Baseline_GRU", "BiLSTM_GRU", "Unidirectional vs Bidirectional Encoder",
         "Bidirectional captures context from both past and future tokens"),
        ("Baseline_GRU", "GRU_Attention", "Without vs With Attention",
         "Attention allows decoder to focus on relevant encoder states"),
        ("Baseline_GRU", "Transformer_GRU", "RNN vs Transformer Encoder",
         "Transformer uses self-attention for parallel processing"),
        ("GRU_Attention", "Transformer_GRU", "RNN Attention vs Transformer",
         "Different attention mechanisms: encoder-decoder vs self-attention"),
    ]
    
    for model1, model2, comparison_name, description in comparisons:
        if model1 in results and model2 in results:
            r1_diff = results[model2]['rouge1_fmeasure'].item() - results[model1]['rouge1_fmeasure'].item()
            r2_diff = results[model2]['rouge2_fmeasure'].item() - results[model1]['rouge2_fmeasure'].item()
            
            winner = model2 if r1_diff > 0 else model1
            
            print(f"\n{'-'*80}")
            print(f"Comparison: {comparison_name}")
            print(f"Description: {description}")
            print(f"{'-'*80}")
            print(f"   {model1.replace('_', ' ')}:")
            print(f"      Rouge-1 F1: {results[model1]['rouge1_fmeasure'].item():.4f}")
            print(f"      Rouge-2 F1: {results[model1]['rouge2_fmeasure'].item():.4f}")
            print(f"   {model2.replace('_', ' ')}:")
            print(f"      Rouge-1 F1: {results[model2]['rouge1_fmeasure'].item():.4f}")
            print(f"      Rouge-2 F1: {results[model2]['rouge2_fmeasure'].item():.4f}")
            print(f"\n   Difference (Model2 - Model1):")
            print(f"      Rouge-1 F1: {r1_diff:+.4f} ({r1_diff/results[model1]['rouge1_fmeasure'].item()*100:+.2f}%)")
            print(f"      Rouge-2 F1: {r2_diff:+.4f} ({r2_diff/results[model1]['rouge2_fmeasure'].item()*100:+.2f}%)")
            print(f"\n   → Winner: {winner.replace('_', ' ')}")

pairwise_comparison(results)

## Visualization: Detailed Performance Charts

In [ ]:
# Advanced Visualization
def create_detailed_visualizations(results):
    """Create comprehensive visualizations for the report."""
    
    models = list(results.keys())
    model_labels = [m.replace('_', '\n') for m in models]
    
    # Extract all metrics
    r1_f1 = [results[m]['rouge1_fmeasure'].item() for m in models]
    r1_prec = [results[m]['rouge1_precision'].item() for m in models]
    r1_rec = [results[m]['rouge1_recall'].item() for m in models]
    r2_f1 = [results[m]['rouge2_fmeasure'].item() for m in models]
    r2_prec = [results[m]['rouge2_precision'].item() for m in models]
    r2_rec = [results[m]['rouge2_recall'].item() for m in models]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Plot 1: Rouge-1 All Metrics
    ax1 = axes[0, 0]
    x = np.arange(len(models))
    width = 0.25
    ax1.bar(x - width, r1_f1, width, label='F1', color='#2ecc71')
    ax1.bar(x, r1_prec, width, label='Precision', color='#3498db')
    ax1.bar(x + width, r1_rec, width, label='Recall', color='#e74c3c')
    ax1.set_ylabel('Score')
    ax1.set_title('Rouge-1 Scores by Model')
    ax1.set_xticks(x)
    ax1.set_xticklabels(model_labels, fontsize=8)
    ax1.legend()
    ax1.set_ylim(0, 1)
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Rouge-2 All Metrics
    ax2 = axes[0, 1]
    ax2.bar(x - width, r2_f1, width, label='F1', color='#2ecc71')
    ax2.bar(x, r2_prec, width, label='Precision', color='#3498db')
    ax2.bar(x + width, r2_rec, width, label='Recall', color='#e74c3c')
    ax2.set_ylabel('Score')
    ax2.set_title('Rouge-2 Scores by Model')
    ax2.set_xticks(x)
    ax2.set_xticklabels(model_labels, fontsize=8)
    ax2.legend()
    ax2.set_ylim(0, 1)
    ax2.grid(axis='y', alpha=0.3)
    
    # Plot 3: F1 Score Comparison (Horizontal Bar)
    ax3 = axes[1, 0]
    y_pos = np.arange(len(models))
    ax3.barh(y_pos - 0.2, r1_f1, 0.4, label='Rouge-1 F1', color='steelblue')
    ax3.barh(y_pos + 0.2, r2_f1, 0.4, label='Rouge-2 F1', color='coral')
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels([m.replace('_', ' ') for m in models])
    ax3.set_xlabel('F1 Score')
    ax3.set_title('F1 Score Comparison')
    ax3.legend(loc='lower right')
    ax3.set_xlim(0, 1)
    ax3.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (v1, v2) in enumerate(zip(r1_f1, r2_f1)):
        ax3.text(v1 + 0.01, i - 0.2, f'{v1:.3f}', va='center', fontsize=8)
        ax3.text(v2 + 0.01, i + 0.2, f'{v2:.3f}', va='center', fontsize=8)
    
    # Plot 4: Radar/Spider Chart (simplified as grouped comparison)
    ax4 = axes[1, 1]
    
    # Calculate improvement over baseline
    baseline_r1 = results['Baseline_GRU']['rouge1_fmeasure'].item()
    baseline_r2 = results['Baseline_GRU']['rouge2_fmeasure'].item()
    
    improvements_r1 = [(results[m]['rouge1_fmeasure'].item() - baseline_r1) / baseline_r1 * 100 for m in models]
    improvements_r2 = [(results[m]['rouge2_fmeasure'].item() - baseline_r2) / baseline_r2 * 100 for m in models]
    
    colors = ['green' if x >= 0 else 'red' for x in improvements_r1]
    ax4.barh(y_pos - 0.2, improvements_r1, 0.4, label='Rouge-1 Improvement', color=['#27ae60' if x >= 0 else '#e74c3c' for x in improvements_r1], alpha=0.7)
    ax4.barh(y_pos + 0.2, improvements_r2, 0.4, label='Rouge-2 Improvement', color=['#2980b9' if x >= 0 else '#c0392b' for x in improvements_r2], alpha=0.7)
    ax4.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels([m.replace('_', ' ') for m in models])
    ax4.set_xlabel('Improvement over Baseline (%)')
    ax4.set_title('Percentage Improvement vs Baseline')
    ax4.legend(loc='lower right')
    ax4.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('detailed_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Visualization saved as 'detailed_analysis.png'")

create_detailed_visualizations(results)

## Explanation of Findings

This section provides theoretical explanations for the observed results.

In [ ]:
# Generate Explanation of Findings
def generate_findings_explanation(results):
    """Generate detailed explanations based on experimental results."""
    
    print("\n" + "="*100)
    print("                              EXPLANATION OF FINDINGS")
    print("="*100)
    
    # Get scores for analysis
    baseline_r1 = results['Baseline_GRU']['rouge1_fmeasure'].item()
    baseline_r2 = results['Baseline_GRU']['rouge2_fmeasure'].item()
    
    explanations = {
        'Baseline_GRU': {
            'name': 'Baseline: GRU Encoder + GRU Decoder',
            'description': '''
The baseline model uses GRU (Gated Recurrent Unit) for both encoder and decoder.

Key Characteristics:
• GRU has two gates: reset gate and update gate
• Simpler than LSTM with fewer parameters
• Encodes the entire input sequence into a single context vector
• Decoder generates output based solely on this fixed-size context vector

Limitations:
• Information bottleneck: entire input compressed into one vector
• May struggle with long sequences
• No direct access to individual encoder states during decoding
'''
        },
        'LSTM_LSTM': {
            'name': 'LSTM Encoder + LSTM Decoder',
            'description': '''
This model replaces GRU with LSTM (Long Short-Term Memory) units.

Key Differences from GRU:
• LSTM has three gates: input, forget, and output gates
• Includes explicit cell state for long-term memory
• Better at capturing long-range dependencies
• More parameters → potentially better expressiveness

Expected Impact:
• May improve performance on longer sequences
• Cell state helps maintain relevant information over time
• Slightly slower training due to more parameters
'''
        },
        'BiLSTM_GRU': {
            'name': 'Bidirectional LSTM Encoder + GRU Decoder',
            'description': '''
This model uses a bidirectional LSTM encoder while keeping GRU decoder.

Key Characteristics:
• Processes input in both forward and backward directions
• Captures context from both past and future tokens
• Concatenates forward and backward hidden states
• Provides richer representation of each input token

Expected Impact:
• Better understanding of context from both directions
• Particularly helpful for translation where word order differs
• May improve handling of syntactic structures
'''
        },
        'GRU_Attention': {
            'name': 'GRU Encoder + GRU Decoder with Attention',
            'description': '''
This model adds Bahdanau-style attention mechanism between encoder and decoder.

Attention Mechanism Details:
• Query: decoder hidden state s_t
• Keys/Values: all encoder hidden states h_i
• Score function: additive (Bahdanau): v^T * tanh(W_s*s_t + W_h*h_i)
• Attention weights: softmax(scores)
• Context vector: weighted sum of encoder states

Expected Impact:
• Eliminates information bottleneck problem
• Decoder can directly access all encoder states
• Different encoder states get different attention at each decoding step
• Particularly effective for translation tasks
• Provides interpretable alignment between source and target
'''
        },
        'Transformer_GRU': {
            'name': 'Transformer Encoder + GRU Decoder',
            'description': '''
This model replaces RNN encoder with Transformer encoder.

Key Characteristics:
• Self-attention mechanism for encoding
• Parallel processing of all input tokens
• Positional encoding for sequence order information
• Multi-head attention captures different aspects of relationships
• Mean pooling of all token representations as sentence embedding

Expected Impact:
• Better at capturing global dependencies
• May require more data for optimal performance
• Faster training due to parallelization
• Different inductive bias compared to RNNs
'''
        }
    }
    
    for model_key, info in explanations.items():
        if model_key in results:
            r1 = results[model_key]['rouge1_fmeasure'].item()
            r2 = results[model_key]['rouge2_fmeasure'].item()
            r1_diff = ((r1 - baseline_r1) / baseline_r1) * 100
            r2_diff = ((r2 - baseline_r2) / baseline_r2) * 100
            
            print(f"\n{'='*80}")
            print(f"MODEL: {info['name']}")
            print(f"{'='*80}")
            print(info['description'])
            print(f"\nActual Results:")
            print(f"   Rouge-1 F1: {r1:.4f} ({r1_diff:+.2f}% vs baseline)")
            print(f"   Rouge-2 F1: {r2:.4f} ({r2_diff:+.2f}% vs baseline)")

generate_findings_explanation(results)

## Final Summary Report

In [ ]:
# Final Summary Report
def generate_final_report(results):
    """Generate a comprehensive final report."""
    
    print("\n" + "="*100)
    print("                              FINAL SUMMARY REPORT")
    print("="*100)
    
    # Get best models
    models = list(results.keys())
    r1_scores = {m: results[m]['rouge1_fmeasure'].item() for m in models}
    r2_scores = {m: results[m]['rouge2_fmeasure'].item() for m in models}
    
    best_r1 = max(r1_scores, key=r1_scores.get)
    best_r2 = max(r2_scores, key=r2_scores.get)
    
    # Calculate average rank
    r1_ranks = {m: sorted(r1_scores.values(), reverse=True).index(r1_scores[m]) + 1 for m in models}
    r2_ranks = {m: sorted(r2_scores.values(), reverse=True).index(r2_scores[m]) + 1 for m in models}
    avg_ranks = {m: (r1_ranks[m] + r2_ranks[m]) / 2 for m in models}
    overall_best = min(avg_ranks, key=avg_ranks.get)
    
    print(f"""
┌────────────────────────────────────────────────────────────────────────────────┐
│                           EXPERIMENT SUMMARY                                    │
├────────────────────────────────────────────────────────────────────────────────┤
│  Total Models Evaluated: {len(models)}                                                       │
│  Evaluation Metric: ROUGE Score (Rouge-1 and Rouge-2)                          │
│  Dataset: French-English Translation (Tatoeba)                                 │
│  Training Epochs: 5                                                            │
└────────────────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────────────────┐
│                              KEY FINDINGS                                       │
├────────────────────────────────────────────────────────────────────────────────┤
│  Best Rouge-1 F1: {best_r1.replace('_', ' '):<25} Score: {r1_scores[best_r1]:.4f}          │
│  Best Rouge-2 F1: {best_r2.replace('_', ' '):<25} Score: {r2_scores[best_r2]:.4f}          │
│  Overall Best:    {overall_best.replace('_', ' '):<25} Avg Rank: {avg_ranks[overall_best]:.1f}            │
└────────────────────────────────────────────────────────────────────────────────┘
""")
    
    # Print insights
    print("""
┌────────────────────────────────────────────────────────────────────────────────┐
│                           KEY INSIGHTS                                          │
├────────────────────────────────────────────────────────────────────────────────┤
│                                                                                │
│  1. ATTENTION MECHANISM                                                        │
│     The attention mechanism allows the decoder to focus on relevant parts      │
│     of the input sequence at each decoding step, addressing the information    │
│     bottleneck problem inherent in basic seq2seq models.                       │
│                                                                                │
│  2. BIDIRECTIONAL ENCODING                                                     │
│     Bidirectional LSTM captures context from both directions, which can        │
│     improve understanding of the source sentence, especially when word         │
│     order differs between languages.                                           │
│                                                                                │
│  3. LSTM vs GRU                                                                │
│     LSTM's explicit cell state may help with longer sequences, though the      │
│     difference may be minimal for shorter sentences in our filtered dataset.   │
│                                                                                │
│  4. TRANSFORMER ENCODER                                                        │
│     Transformer's self-attention can capture global dependencies but may       │
│     require more data and careful hyperparameter tuning for optimal results.   │
│                                                                                │
└────────────────────────────────────────────────────────────────────────────────┘
""")
    
    # Final ranking table
    print("\n┌────────────────────────────────────────────────────────────────────────────────┐")
    print("│                           FINAL MODEL RANKING                                  │")
    print("├────────────────────────────────────────────────────────────────────────────────┤")
    
    sorted_models = sorted(avg_ranks.keys(), key=lambda x: avg_ranks[x])
    for rank, model in enumerate(sorted_models, 1):
        medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
        print(f"│  {medal} Rank {rank}: {model.replace('_', ' '):<30} R1: {r1_scores[model]:.4f}  R2: {r2_scores[model]:.4f} │")
    
    print("└────────────────────────────────────────────────────────────────────────────────┘")

generate_final_report(results)

In [ ]:
# Export Results to CSV for Report
def export_results_to_csv(results):
    """Export all results to CSV files for inclusion in reports."""
    
    # Full results table
    full_data = []
    for model_name, scores in results.items():
        full_data.append({
            'Model': model_name.replace('_', ' '),
            'Rouge-1 F1': round(scores['rouge1_fmeasure'].item(), 4),
            'Rouge-1 Precision': round(scores['rouge1_precision'].item(), 4),
            'Rouge-1 Recall': round(scores['rouge1_recall'].item(), 4),
            'Rouge-2 F1': round(scores['rouge2_fmeasure'].item(), 4),
            'Rouge-2 Precision': round(scores['rouge2_precision'].item(), 4),
            'Rouge-2 Recall': round(scores['rouge2_recall'].item(), 4),
        })
    
    df_full = pd.DataFrame(full_data)
    df_full.to_csv('results_full.csv', index=False)
    
    # Summary table (F1 only)
    summary_data = []
    baseline_r1 = results['Baseline_GRU']['rouge1_fmeasure'].item()
    baseline_r2 = results['Baseline_GRU']['rouge2_fmeasure'].item()
    
    for model_name, scores in results.items():
        r1 = scores['rouge1_fmeasure'].item()
        r2 = scores['rouge2_fmeasure'].item()
        summary_data.append({
            'Model': model_name.replace('_', ' '),
            'Rouge-1 F1': round(r1, 4),
            'Rouge-2 F1': round(r2, 4),
            'R1 Improvement (%)': round(((r1 - baseline_r1) / baseline_r1) * 100, 2),
            'R2 Improvement (%)': round(((r2 - baseline_r2) / baseline_r2) * 100, 2),
        })
    
    df_summary = pd.DataFrame(summary_data)
    df_summary.to_csv('results_summary.csv', index=False)
    
    print("✓ Results exported to:")
    print("   - results_full.csv (all metrics)")
    print("   - results_summary.csv (F1 scores with improvements)")
    print("\nYou can use these CSV files in your report!")
    
    return df_full, df_summary

df_full, df_summary = export_results_to_csv(results)
print("\n" + "="*60)
print("Summary Table (for copy-paste into report):")
print("="*60)
display(df_summary)

## Conclusions

### Summary of Experiments

This project implemented and compared five different sequence-to-sequence architectures for French-to-English translation:

| Task | Model | Encoder | Decoder | Key Feature |
|------|-------|---------|---------|-------------|
| 2 | Baseline | GRU | GRU | Standard seq2seq |
| 3 | LSTM | LSTM | LSTM | Cell state for long-term memory |
| 4 | BiLSTM | Bi-LSTM | GRU | Bidirectional context encoding |
| 5 | Attention | GRU | GRU + Attention | Bahdanau attention mechanism |
| 6 | Transformer | Transformer | GRU | Self-attention encoding |

### Key Findings

1. **Attention Mechanism**: Adding attention typically improves translation quality by allowing the decoder to directly access relevant encoder states at each timestep, solving the information bottleneck problem.

2. **Bidirectional Encoding**: Bi-LSTM captures context from both directions, which is particularly useful for translation where word order may differ between languages.

3. **LSTM vs GRU**: Both architectures perform similarly on this task. LSTM's explicit cell state may provide marginal benefits for longer sequences.

4. **Transformer Encoder**: While powerful, the Transformer may require more training data and careful hyperparameter tuning to outperform attention-based RNN models on smaller datasets.

### Recommendations

- For **production systems**: Use attention-based models for best translation quality
- For **limited computational resources**: GRU-based models offer good balance of speed and performance
- For **longer sequences**: Consider Bi-LSTM encoder with attention

### Future Work

- Experiment with different attention mechanisms (Luong vs Bahdanau)
- Try full Transformer architecture (encoder + decoder)
- Implement beam search for decoding
- Train on larger datasets for more robust comparisons